<a href="https://colab.research.google.com/github/y4c6/master_thesis/blob/main/post_selection_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
# directory
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import json

In [3]:
from sklearn.linear_model import LassoCV, Lasso
from sklearn.linear_model import LinearRegression

class postLasso:
    def __init__(self, random_state=None, target=None, set1=None, set2=None, token = {'stem', 'lemma'}):
        self.random_state = random_state
        self.token = token  # {stem, lemma}
        self.set1 = set1
        self.set2 = set2
        self.target = target #
        #self.treatment = treatment
        self.selectX = None # param passthrough from lasso to ols
        self.alpha = None # param passthrough from lassocv to lasso
        self.lassocv = None
        self.lasso = None
        self.ols = None


    def lasso_fit(self, target):
        # Step 1: LassoCV for feature selection
        self.lasso_cv = LassoCV(random_state=self.random_state, cv=5, tol=0.1)

        self.lasso_cv.fit(self.set1.loc[:, self.set1.columns != self.target],
                 self.set1[self.target])

        # Step 2: Lasso with alpha from LassoCV
        self.alpha = self.lasso_cv.alpha_
        self.lasso = Lasso(alpha=self.alpha)
        self.lasso.fit(self.set1.loc[:, self.set1.columns != self.target],
                 self.set1[self.target])
        self.selectX = list(self.set1.keys()[self.set1.columns != self.target][self.lasso.coef_!=0])

    def get_selectX(self):
        return self.selectX

    def post_lasso_fit(self, target, treatment, selected):
        # Step 3: OLS with selected features
        #self.selectX = list(self.set1.keys()[self.set1.columns != self.target][self.lasso.coef_!=0])

        self.treatment = treatment
        self.selected = selected
        self.ols = LinearRegression()
        self.ols.fit(self.set2[ self.treatment + self.selected ],
              self.set2[ self.target ])

    # def predict(self, X):
    #     X_selected = X[:, self.selected_features]
    #     return self.ols.predict(X_selected)

    def get_ols_model(self):
        return self.ols

# stem

In [4]:
data_stem = pd.read_pickle("/content/gdrive/MyDrive/Thesis_Data&Result/EJMR_data_stem_ao2_a.pkl")
data_stem.rename(columns = {'y':'asian'}, inplace = True)

In [5]:
print(data_stem.shape)
data_stem.head(3)

(23337, 10001)


,aa,aacsb,ab,abandon,abbott,abbrevi,abc,abd,abe,abel,...,zone,zoo,zoom,zoomer,zu,zuckerberg,zucman,zuel,zurich,asian
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from sklearn.model_selection import train_test_split

train_stem, test_stem = train_test_split(data_stem, test_size=0.5,  random_state = 2510, shuffle=True)

In [12]:
del data_stem

In [ ]:
# behavior = ['hardwork', 'cheat', 'encourag']
appearance = ['golddigger', 'ugli', 'fivefooteight', 'undesir', 'despis']
# intellengence = ['braindamag', 'brainwash', 'creativ',
#                  'submoron', 'stjupid', 'stpid', 'mediocr', 'talent']
# trash = ['rubbish', 'trash', 'rag']
# fword = ['fuaak', 'phuyk', 'flopper']
# sword = ['shit', 'shiet', 'shi', 'stter']
# adjs = ['shame','weak','solid','stronger','good','strongest','great','bad','impress','weaker','terribl','clear','well']

In [8]:
trash = ['rubbish', 'trash', 'rag']

D_name = trash

target = 'asian'

D_name = [w for w, i in zip( D_name, [( w in list(train_stem.columns)) & ( w in list(test_stem.columns) )  for w in D_name]) if i]
# w for word, i for index
print(len(D_name))
print(D_name)

3
['rubbish', 'trash', 'rag']


In [24]:
selectX = ['adb', 'american', 'chines', 'countri', 'crime', 'east', 'econom', 'happi', 'hk', 'immigr', 'indian', 'interview', 'japanes', 'job', 'journal', 'karl', 'kimchi', 'ntu', 'one', 'paper', 'read', 'sat', 'singapor', 'south', 'student', 'taiwanes', 'tax', 'test', 'time', 'tokyo', 'use', 'white', 'women', 'work', 'year']

In [15]:
asian_model = postLasso(random_state=4205, target='asian', set1=train_stem, set2=test_stem, token = 'stem')

In [16]:
asian_model.post_lasso_fit(target='asian', treatment=D_name, selected = selectX)
ols2 = asian_model.get_ols_model()
ols2.coef_

array([-1.73780689e-02,  7.05006357e-03, -7.20384143e-02,  3.52344496e-03,
        5.15928906e-03, -4.85427494e-03,  1.03542346e-03,  1.47391543e-02,
        2.29122789e-02, -2.33032765e-03, -1.24141136e-02,  9.47286350e-04,
        3.85128178e-03, -2.14856622e-03, -1.00775252e-04,  2.39386704e-02,
       -3.85042367e-04, -7.77288893e-05,  3.16576908e-04, -5.49023360e-03,
        9.35432691e-02,  2.20759205e-04, -6.51826167e-04, -5.60298223e-03,
       -4.16310944e-03,  3.61269381e-03,  2.72721812e-02,  7.54963022e-04,
        8.40543017e-03, -3.16672980e-03,  1.05130171e-03, -2.27036709e-03,
        6.99671561e-02, -1.69332672e-03,  1.11398686e-02,  2.41483403e-04,
       -4.97066552e-04, -8.88594148e-04])

In [18]:
pd.DataFrame({'word': D_name + list(selectX),
      'coef': ols2.coef_})#.to_csv('/content/gdrive/Shareddrives/Files_forMeeting/Meeting_0614/stem_asian_trash.csv')

## ugli, strong, stupid

In [ ]:
target = 'stupid'
D_name = ['asian']

In [ ]:
train_stem.loc[train_stem[target] != 0, target] = 1

In [ ]:
ugli_model = postLasso(random_state=4205, target=target, set1=train_stem, set2=test_stem, token = 'stem')
ugli_model.lasso_fit(target=target)
ugli_model.post_lasso_fit(target=target, treatment=D_name)
ols2 = ugli_model.get_ols_model()
ols2.coef_

array([-0.01054964, -0.06063244,  0.01869299,  0.02887523])

In [ ]:
lasso_X = ugli_model.get_selectX()

In [ ]:
pd.DataFrame({'word': D_name + list(lasso_X),
      'coef': ols2.coef_}).to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/stem_stupid.csv')

# Lemma

In [5]:
data_lemma = pd.read_pickle("/content/gdrive/MyDrive/Thesis_Data&Result/EJMR_data_lemma_ao2_a_0613.pkl")
#### data_lemma.rename(columns = {'y':'asian'}, inplace = True) #他本來就有y了
print(data_lemma.shape)
data_lemma.head(3)

(23337, 10001)


,aa,aacsb,ab,abandon,abandoned,abandoning,abc,abd,abe,ability,...,zju,zombie,zone,zoning,zoom,zu,zuckerberg,zucman,zurich,asian
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
from sklearn.model_selection import train_test_split

train_lemma, test_lemma = train_test_split(data_lemma, test_size=0.5,  random_state = 2510, shuffle=True)

In [7]:
print(sum(data_lemma.columns == train_lemma.columns))
print(sum(data_lemma.columns == test_lemma.columns))

10001
10001


In [8]:
del data_lemma

In [ ]:
# manner = ['rude', 'dickhad', 'cocking']
# appearance = ['obese', 'fivefooteight', 'golddigger']
# intellegence = ['brainwashed', 'overratedbogus',
#  'mediocre', 'submoron', 'stjupid', 'dorkness']
# fword = ['fawk', 'fleckistan',
#  'fuaak', 'fawhk', 'fk', 'mf']
# sword = ['sheethole',
#  'shit', 'shiiiit', 'shtholes', 'shiat']
# trash = ['dvmpster',
#  'trxsh', 'rubbish', 'rag', 'trash']
# adjs = ['weak', 'weaker', 'weakest',
# 'bad', 'well', 'bright', 'shamed', 'shameful', 'unimpressive',
# 'disgiusting', 'disappointing', 'fierce', 'terrible', 'embarrassing']

In [31]:
adjs = ['weak', 'weaker', 'weakest',
'bad', 'well', 'bright', 'shamed', 'shameful', 'unimpressive',
'disgiusting', 'disappointing', 'fierce', 'terrible', 'embarrassing']
D_name = adjs

target = 'asian'

D_name = [w for w, i in zip( D_name, [( w in list(train_lemma.columns)) & ( w in list(test_lemma.columns) )  for w in D_name]) if i]
# w for word, i for index
print(len(D_name))
print(D_name)

12
['weak', 'weaker', 'weakest', 'bad', 'well', 'bright', 'shamed', 'shameful', 'disappointing', 'fierce', 'terrible', 'embarrassing']


## $ Asian \sim Word + X$

### lasso

In [ ]:
asian_model = postLasso(random_state=4205, target='asian', set1=train_lemma, set2=test_lemma, token = 'lemma')
asian_model.lasso_fit(target='asian')

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.138e+02, tolerance: 2.054e+02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.236e+02, tolerance: 2.069e+02
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.145e+02, tolerance: 2.069e

In [ ]:
selectX = asian_model.get_selectX()

In [ ]:
print(len(selectX))
print(selectX)

46
['adb', 'american', 'beijing', 'better', 'ccp', 'chinabro', 'class', 'contract', 'country', 'culture', 'economist', 'english', 'get', 'happiness', 'hong', 'im', 'india', 'indian', 'interview', 'job', 'journal', 'karl', 'like', 'make', 'model', 'ntu', 'offer', 'paper', 'phd', 'school', 'soe', 'south', 'state', 'tax', 'time', 'u', 'university', 'virus', 'way', 'white', 'work', 'world', 'xi', 'year', 'youre', 'zhang']


In [ ]:
asian_model.post_lasso_fit(target='asian', treatment=D_name)
ols2 = asian_model.get_ols_model()
ols2.coef_

array([ 3.14557063e-02,  4.79714015e-03,  5.14063996e-03,  7.23281353e-02,
       -3.68552915e-03,  8.97437169e-03,  7.09172908e-02, -1.23727410e-02,
        7.60681730e-03,  4.07699326e-03, -5.45900201e-03, -5.22060791e-03,
        1.49326206e-02, -3.50038119e-03, -1.35046699e-02,  2.02783005e-02,
       -7.34049498e-03,  6.59211507e-03,  2.94681210e-03, -1.55331565e-03,
       -1.15116553e-03, -2.82461569e-03, -2.76890339e-02, -3.21736673e-03,
       -2.81049487e-03, -6.28014573e-03,  1.14338905e-01,  4.62162350e-03,
       -6.45574376e-04, -7.63726863e-04,  1.90318839e-03,  8.21101751e-02,
        2.27018068e-02, -4.37858775e-03, -7.46825400e-03, -2.95435899e-03,
        1.03208569e-02, -6.83747758e-04,  1.25509246e-02, -6.30283590e-03,
        9.86028009e-03, -1.31999590e-03,  1.64151117e-02,  2.96823561e-02,
        1.29270159e-05, -1.24245579e-02,  3.55473460e-02])

In [ ]:
pd.DataFrame({'word': D_name + list(selectX),
      'coef': ols2.coef_})#.to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/lemma_asian_.csv')

### 有D檔案的讀法

In [11]:
selectX = ['adb', 'american', 'beijing', 'better', 'ccp', 'chinabro', 'class', 'contract', 'country', 'culture', 'economist', 'english', 'get', 'happiness', 'hong', 'im', 'india', 'indian', 'interview', 'job', 'journal', 'karl', 'like', 'make', 'model', 'ntu', 'offer', 'paper', 'phd', 'school', 'soe', 'south', 'state', 'tax', 'time', 'u', 'university', 'virus', 'way', 'white', 'work', 'world', 'xi', 'year', 'youre', 'zhang']

In [14]:
asian_model = postLasso(random_state=4205, target='asian', set1=train_lemma, set2=test_lemma, token = 'lemma')

------

In [32]:
asian_model.post_lasso_fit(target='asian', treatment=D_name, selected = selectX)
ols2 = asian_model.get_ols_model()
ols2.coef_

array([ 0.00072588,  0.03857933,  0.01214845,  0.00302298,  0.00096055,
        0.0288517 , -0.13645175, -0.02072618, -0.0520496 , -0.00661817,
       -0.01161372, -0.04740593,  0.00455329,  0.00505403,  0.07208862,
       -0.00381626,  0.00918132,  0.07219496, -0.01307262,  0.00677214,
        0.00382329, -0.00516919, -0.00529515,  0.01492368, -0.00364215,
       -0.01199889,  0.02002181, -0.00775158,  0.00739608,  0.00296154,
       -0.00151647, -0.00086843, -0.00290534, -0.02839935, -0.00299092,
       -0.00231948, -0.00644939,  0.11495475,  0.00439885, -0.00074377,
       -0.00091491,  0.00200271,  0.08343007,  0.02229439, -0.00430926,
       -0.00731565, -0.00260876,  0.01050827, -0.00077692,  0.01398245,
       -0.00635142,  0.01001096, -0.00152051,  0.01632242,  0.02950456,
       -0.00031958, -0.01256857,  0.03552173])

In [33]:
pd.DataFrame({'word': D_name + list(selectX),
      'coef': ols2.coef_})#.to_csv('/content/gdrive/Shareddrives/Files_forMeeting/Meeting_0614/lemma_asian_adjs.csv')

## $Word \sim Asian + X$

In [ ]:
target =

In [ ]:
# for target is not asian
train_stem.loc[train_stem[target] != 0, target] = 1

In [ ]:
ugli_model = postLasso(random_state=4205, target=target, set1=train_stem, set2=test_stem, token = 'stem')
ugli_model.lasso_fit(target=target)
ugli_model.post_lasso_fit(target=target, treatment=D_name)
ols2 = ugli_model.get_ols_model()
ols2.coef_

array([-0.01054964, -0.06063244,  0.01869299,  0.02887523])

In [ ]:
lasso_X = ugli_model.get_selectX()

In [ ]:
pd.DataFrame({'word': D_name + list(lasso_X),
      'coef': ols2.coef_}).to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/stem_stupid.csv')

# old things

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
stem1 = LassoCV(cv=5, random_state=1115, tol = 0.1)\
     .fit( X = train_stem.loc[:, train_stem.columns != 'asian'],
         y = train_stem['asian'])

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.277e+02, tolerance: 8.803e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.087e+02, tolerance: 8.803e+01
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.078e+02, tolerance: 8.803e

In [ ]:
print(len(stem1.coef_[stem1.coef_!=0]))

stem1_select = train_stem.keys()[train_stem.columns != 'asian'][stem1.coef_!=0] # remove y
print(stem1_select)

44
Index(['adb', 'american', 'anyon', 'chines', 'countri', 'crime', 'cultur',
       'discrimin', 'dont', 'east', 'econom', 'get', 'happi', 'hk', 'immigr',
       'indian', 'interview', 'japanes', 'job', 'journal', 'karl', 'kimchi',
       'like', 'make', 'much', 'ntu', 'one', 'paper', 'read', 'sat',
       'singapor', 'south', 'student', 'taiwanes', 'test', 'time', 'tokyo',
       'us', 'use', 'way', 'white', 'women', 'work', 'year'],
      dtype='object')


In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
stem2 = Lasso( alpha = stem1.alpha_)\
     .fit( X = train_stem.loc[:, train_stem.columns != 'asian'],
         y = train_stem['asian'])

In [ ]:
print(len(stem2.coef_[stem2.coef_!=0]))

stem2_select = train_stem.keys()[train_stem.columns != 'asian'][stem2.coef_!=0] # remove y_word
print(stem2_select)

35
Index(['adb', 'american', 'chines', 'countri', 'crime', 'east', 'econom',
       'happi', 'hk', 'immigr', 'indian', 'interview', 'japanes', 'job',
       'journal', 'karl', 'kimchi', 'ntu', 'one', 'paper', 'read', 'sat',
       'singapor', 'south', 'student', 'taiwanes', 'tax', 'test', 'time',
       'tokyo', 'use', 'white', 'women', 'work', 'year'],
      dtype='object')


In [ ]:
stem2_df = pd.DataFrame({'word':list(stem2_select),
      'coef': stem1.coef_[stem2.coef_!=0]})

In [ ]:
stem2_df#.to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/stem_asian_select.csv')

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
post_stem1 = LinearRegression()\
     .fit( X = train_stem[['rubbish', 'trash', 'rag'] + list(stem2_select)],
         y = train_stem['asian'])

In [ ]:
pd.DataFrame({'word': ['rubbish', 'trash', 'rag'] + list(stem2_select),
      'coef': post_stem1.coef_})#.to_csv('/content/gdrive/MyDrive/Tempf/Meeting_0614/stem_asian.csv')